In [1]:
!pip install -qU "langchain[together]" langchain-huggingface


In [2]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

model = init_chat_model(model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free", model_provider="together")


In [3]:
def chat_bot():
    user_input = input("You : ")
    while True:
        if user_input in ["bye"]:
            break
        else:
            model.invoke([HumanMessage(content=user_input)])


In [4]:
chat_bot()


KeyboardInterrupt: 

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
import faiss


In [ ]:
index = faiss.read_index(FAISS_INDEX_PATH)

with open(META_PATH, "r", encoding="utf-8") as f:
    metadata = [line.strip().split("\t") for line in f.readlines()] 

with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
    chunks_data = json.load(f) 

print("FAISS index, metadata, and chunks restored successfully!")


In [ ]:
embeddings = HuggingFaceEmbeddings("bert-base-uncased")


In [11]:
from transformers import pipeline
import numpy as np

classifier = pipeline('zero-shot-classification', model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
labels = ["Python Programming", "Not Python Programming"]

query = "I love python species"
result = classifier(query, candidate_labels=labels)
index = np.argmax(result['scores'])
print(result['labels'][index])


Device set to use mps:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Python Programming
